# Modeling

## Hans

## Annie: KNN and gradient boosting classifier

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import metrics
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline

#from skopt import BayesSearchCV
import pickle
import datetime
import streamlit as st

In [7]:
df = pd.read_csv('../Data/salary_cleaned.csv')
df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,year,month,year_month,timestamp_3mos,year_month_3mos,state_short,inflation_rate,inflation_rate_3mos,state,employment_rate,employment_rate_3mos
0,2018-06-03 13:58:20,Yahoo,IC2,Software Engineer,160.5,"Sunnyvale, CA",0.58,0.58,Full Stack,2018,6,2018-06,2018-03-03 13:58:20,2018-03,CA,0.029,0.024,California,0.957660,0.956797
1,2018-06-04 20:28:22,Facebook,E3,Software Engineer,165.0,"Seattle, WA",1.00,1.00,Full Stack,2018,6,2018-06,2018-03-04 20:28:22,2018-03,WA,0.029,0.024,Washington,0.955998,0.954978
2,2018-06-05 00:56:33,VmWare,Senior MTS,Software Engineer,218.0,"Palo Alto, CA",8.00,1.00,Distributed Systems (Back-End),2018,6,2018-06,2018-03-05 00:56:33,2018-03,CA,0.029,0.024,California,0.957660,0.956797
3,2018-06-05 01:19:05,Uber,L4,Software Engineer,240.0,"San Francisco, CA",3.00,0.00,Web Development (Front-End),2018,6,2018-06,2018-03-05 01:19:05,2018-03,CA,0.029,0.024,California,0.957660,0.956797
4,2018-06-05 07:13:17,Capital One,Master Software Engineer,Software Engineer,196.0,"New York, NY",8.00,2.00,iOS,2018,6,2018-06,2018-03-05 07:13:17,2018-03,NY,0.029,0.024,New York,0.959053,0.955962


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24496 entries, 0 to 24495
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                24496 non-null  object 
 1   company                  24496 non-null  object 
 2   level                    24496 non-null  object 
 3   title                    24496 non-null  object 
 4   totalyearlycompensation  24496 non-null  float64
 5   location                 24496 non-null  object 
 6   yearsofexperience        24496 non-null  float64
 7   yearsatcompany           24496 non-null  float64
 8   tag                      24496 non-null  object 
 9   year                     24496 non-null  int64  
 10  month                    24496 non-null  int64  
 11  year_month               24496 non-null  object 
 12  timestamp_3mos           24496 non-null  object 
 13  year_month_3mos          24496 non-null  object 
 14  state_short           

In [9]:
# convert year and month from integer to string
df['year'] = df['year'].astype(str)
df['month'] = df['month'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24496 entries, 0 to 24495
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                24496 non-null  object 
 1   company                  24496 non-null  object 
 2   level                    24496 non-null  object 
 3   title                    24496 non-null  object 
 4   totalyearlycompensation  24496 non-null  float64
 5   location                 24496 non-null  object 
 6   yearsofexperience        24496 non-null  float64
 7   yearsatcompany           24496 non-null  float64
 8   tag                      24496 non-null  object 
 9   year                     24496 non-null  object 
 10  month                    24496 non-null  object 
 11  year_month               24496 non-null  object 
 12  timestamp_3mos           24496 non-null  object 
 13  year_month_3mos          24496 non-null  object 
 14  state_short           

In [10]:
df['location'].value_counts(normalize=True)

Seattle, WA          0.161169
San Francisco, CA    0.136920
New York, NY         0.078829
Redmond, WA          0.065072
Sunnyvale, CA        0.049110
                       ...   
Berlin, CT           0.000041
Aliso Viejo, CA      0.000041
West Chester, PA     0.000041
Iselin, NJ           0.000041
Bowling Green, OH    0.000041
Name: location, Length: 447, dtype: float64

In [11]:
# create 
features_all = ['company', 'title', 'location', 'yearsofexperience', 'yearsatcompany', 'year', 'month', 'year_month', 'state_short', 'inflation_rate', 'inflation_rate_3mos', 'employment_rate', 'employment_rate_3mos']
features_short = ['company', 'title', 'yearsofexperience', 'yearsatcompany', 'year', 'month', 'state', 'inflation_rate', 'inflation_rate_3mos', 'employment_rate', 'employment_rate_3mos']

X = df[features_short]
y = df['totalyearlycompensation']
X.shape, y.shape

((24496, 11), (24496,))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((18372, 11), (18372,), (6124, 11), (6124,))

In [13]:
X_train.head()

,company,title,yearsofexperience,yearsatcompany,year,month,state,inflation_rate,inflation_rate_3mos,employment_rate,employment_rate_3mos
19069,Google,Product Manager,15.0,1.0,2020,7,Washington,0.010,0.003,0.898081,0.837230
3156,Amazon,Software Engineer,5.0,5.0,2019,3,Washington,0.019,0.019,0.955238,0.954989
14183,Twilio,Software Engineer,4.0,1.0,2020,4,California,0.003,0.025,0.836228,0.961390
13104,Intel,Software Engineer,9.0,9.0,2020,3,Arizona,0.015,0.023,0.938761,0.954570
20679,Cisco,Software Engineer,0.0,0.0,2020,8,California,0.013,0.001,0.888158,0.835571


### Transform data: standardize and one hot encoding

In [78]:
ct = ColumnTransformer([
    ('sc', StandardScaler(), make_column_selector(dtype_include=np.number)),
    #('ohe', OneHotEncoder(handle_unknown='ignore'), make_column_selector(dtype_include=object)),
    #('ohe', OneHotEncoder(handle_unknown='ignore'), ['company','title','year','month','state']),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False), [0,1,6])
    ])

X_train_ct = ct.fit_transform(X_train)
X_test_ct = ct.transform(X_test)

In [45]:
with open('../Models/ColumnTransformer.pkl', 'wb') as f:
    pickle.dump(ct, f)

In [46]:
with open('../Models/ColumnTransformer.pkl', 'rb') as f:
    ct_pkl = pickle.load(f)

In [47]:
test = ct_pkl.transform(X_train)
test.shape

(18372, 1219)

In [79]:
X_train_ct.shape

(18372, 1219)

In [16]:
X_train

,company,title,yearsofexperience,yearsatcompany,year,month,state,inflation_rate,inflation_rate_3mos,employment_rate,employment_rate_3mos
19069,Google,Product Manager,15.0,1.0,2020,7,Washington,0.010,0.003,0.898081,0.837230
3156,Amazon,Software Engineer,5.0,5.0,2019,3,Washington,0.019,0.019,0.955238,0.954989
14183,Twilio,Software Engineer,4.0,1.0,2020,4,California,0.003,0.025,0.836228,0.961390
13104,Intel,Software Engineer,9.0,9.0,2020,3,Arizona,0.015,0.023,0.938761,0.954570
20679,Cisco,Software Engineer,0.0,0.0,2020,8,California,0.013,0.001,0.888158,0.835571
...,...,...,...,...,...,...,...,...,...,...,...
21575,BlackRock,Product Manager,8.0,2.0,2020,8,California,0.013,0.001,0.888158,0.835571
5390,Amazon,Software Engineer,2.0,0.0,2019,7,Washington,0.018,0.020,0.957501,0.955663
860,Splunk,Product Manager,9.0,1.0,2018,9,Washington,0.023,0.029,0.956046,0.955998
15795,Facebook,Software Engineer,5.0,1.0,2020,5,California,0.001,0.023,0.835571,0.961092


In [17]:
ct.get_feature_names_out()

array(['sc__yearsofexperience', 'sc__yearsatcompany',
       'sc__inflation_rate', ..., 'ohe__state_Washington',
       'ohe__state_West Virginia', 'ohe__state_Wisconsin'], dtype=object)

### KNN 

In [14]:
knn = KNeighborsRegressor(n_jobs=-1)

knn_params = {
    'n_neighbors': [3,5,7,9,11,13,15,17,19]
    ,'weights': ['uniform', 'distance']
    ,'p': [1,2]   # this one makes the training time much longer, and didn't improve R2 significantly
}

gs_knn = GridSearchCV(estimator = knn,
                     param_grid = knn_params,
                     cv = 5)

# train the model

print(datetime.datetime.now())

gs_knn.fit(X_train_ct, y_train)

print(datetime.datetime.now())
print()

2022-01-06 13:28:26.415902
2022-01-06 13:31:47.276782



In [15]:
# predict
knn_pred_train = gs_knn.predict(X_train_ct)
knn_pred_test = gs_knn.predict(X_test_ct)

# evaluate
gs_knn_R2_train = gs_knn.score(X_train_ct, y_train)
gs_knn_R2_test = gs_knn.score(X_test_ct, y_test)

print(f'gs_knn_R2_train: {round(gs_knn_R2_train, 4)}')
print(f'gs_knn_R2_test: {round(gs_knn_R2_test, 4)}')

gs_knn_R2_train: 0.9907
gs_knn_R2_test: 0.4762


In [16]:
gs_knn_mse_train = metrics.mean_squared_error(y_train, knn_pred_train)
gs_knn_mse_test = metrics.mean_squared_error(y_test, knn_pred_test)

print(f'gs_knn_mse_train: {round(gs_knn_mse_train, 4)}')
print(f'gs_knn_mse_test: {round(gs_knn_mse_test, 4)}')

gs_knn_mse_train: 158.3478
gs_knn_mse_test: 9172.5505


In [17]:
perf_dict = {
    'R2':[gs_knn_R2_train, gs_knn_R2_test],
    'mse':[gs_knn_mse_train, gs_knn_mse_test]
}

In [18]:
perf_df = pd.DataFrame(perf_dict, index=['Train','Test'])


In [19]:
gs_knn.best_estimator_

KNeighborsRegressor(n_jobs=-1, n_neighbors=13, p=1, weights='distance')

##### Pickle and save the model

In [27]:
with open('../Models/totalcomp_gs_knn.pkl', 'wb') as f:
    pickle.dump(gs_knn, f)

In [28]:
'''
with open('../Models/totalcomp_gs_knn.pkl', 'rb') as f:
    gs_knn = pickle.load(f)
'''

In [22]:
#gs_knn.cv_results_

In [21]:
#pd.DataFrame(gs_knn.cv_results_)

**Observation**: 
1. Comparing the R2 score and mse between the training and testing sets, the model is very overfit.
2. The best model from grid search CV uses 13 nearest neighbors, p=1, and distance as weights.

### Graident Boosting Classifer

#### No grid search

In [54]:
# fit the model
gbr = GradientBoostingRegressor(n_estimators=400,
                                max_depth=3,
                                max_features=800,
                                random_state=42)

print(datetime.datetime.now())

gbr.fit(X_train_ct, y_train)

print(datetime.datetime.now())

2022-01-05 15:52:09.839168
2022-01-05 15:54:41.358522


In [58]:
# predict
gbr_pred_train = gbr.predict(X_train_ct)
gbr_pred_test = gbr.predict(X_test_ct)

# evaluate
gbr_R2_train = gbr.score(X_train_ct, y_train)
gbr_R2_test = gbr.score(X_test_ct, y_test)

gbr_mse_train = metrics.mean_squared_error(y_train, gbr_pred_train)
gbr_mse_test = metrics.mean_squared_error(y_test, gbr_pred_test)

gbr_perf_dict = {
    'R2': [gbr_R2_train, gbr_R2_test],
    'MSE': [gbr_mse_train, gbr_mse_test]
}

gbr_perf_df = pd.DataFrame(gbr_perf_dict, index=['Train','Test'])
gbr_perf_df.T

,Train,Test
R2,0.597249,0.531825
MSE,6834.117462,8198.404325


#### GridSearchCV

##### GridSearch 0 
Model No. 0: killed after running for 19 hours

##### GridSearch 1

In [69]:
# build the grid search for hyperparameters 

gbr = GradientBoostingRegressor(random_state=42)

gbr_params = {
    'n_estimators': [300,400,500],
    #'learning_rate': [0.01, 0.1],
    'max_depth': [5,6,7,8],
    #'min_samples_split': [5,7,10],
    #'min_samples_leaf': [2,3,5],
    'max_features': [100,150,200]
}

gs_gbr = GridSearchCV(gbr,
                     gbr_params,
                     cv=5)

# train

print(datetime.datetime.now())

gs_gbr.fit(X_train_ct, y_train)

print(datetime.datetime.now())

2022-01-05 21:12:19.908880
2022-01-05 23:36:23.269022


In [78]:
# predict

gs_gbr_pred_train = gs_gbr.predict(X_train_ct)
gs_gbr_pred_test = gs_gbr.predict(X_test_ct)

# evaluate

gs_gbr_R2_train = gs_gbr.score(X_train_ct, y_train)
gs_gbr_R2_test = gs_gbr.score(X_test_ct, y_test)

print(f'gs_gbr_R2_train: {round(gs_gbr_R2_train, 4)}')
print(f'gs_gbr_R2_test: {round(gs_gbr_R2_test, 4)}')
print()

gs_gbr_mse_train = metrics.mean_squared_error(y_train, gs_gbr_pred_train)
gs_gbr_mse_test = metrics.mean_squared_error(y_test, gs_gbr_pred_test)

print(f'gs_gbr_mse_train: {round(gs_gbr_mse_train, 4)}')
print(f'gs_gbr_mse_test: {round(gs_gbr_mse_test, 4)}')

gs_gbr_perf_dict = {
    'R2': [gs_gbr_R2_train, gs_gbr_R2_test],
    'MSE': [gs_gbr_mse_train, gs_gbr_mse_test]
}

gs_gbr_perf_df = pd.DataFrame(gs_gbr_perf_dict, index=['Train','Test'])
gs_gbr_perf_df.T

gs_gbr_R2_train: 0.7683
gs_gbr_R2_test: 0.5496

gs_gbr_mse_train: 3932.287
gs_gbr_mse_test: 7887.0258


,Train,Test
R2,0.768261,0.549606
MSE,3932.287030,7887.025790


In [79]:
gs_gbr.best_estimator_

GradientBoostingRegressor(max_depth=7, max_features=200, n_estimators=500,
                          random_state=42)

In [80]:
gs_gbr.best_score_

0.5460109618022845

In [81]:
# pd.DataFrame(gs_gbr.cv_results_)

##### GridSearch 2

In [18]:
# build the grid search for hyperparameters 

gbr2 = GradientBoostingRegressor(random_state=42)

gbr2_params = {
    'n_estimators': [400],
    #'learning_rate': [0.01, 0.1],
    'max_depth': [3,4,5,6],
    #'min_samples_split': [5,7,10],
    #'min_samples_leaf': [2,3,5],
    'max_features': [200, 300]
}

gs_gbr2 = GridSearchCV(gbr2,
                     gbr2_params,
                     cv=5)

# train

print(datetime.datetime.now())

gs_gbr2.fit(X_train_ct, y_train)

print(datetime.datetime.now())

2022-01-06 16:12:38.457989
2022-01-06 16:16:14.737915


In [19]:
# predict

gs_gbr2_pred_train = gs_gbr2.predict(X_train_ct)
gs_gbr2_pred_test = gs_gbr2.predict(X_test_ct)

# evaluate

gs_gbr2_R2_train = gs_gbr2.score(X_train_ct, y_train)
gs_gbr2_R2_test = gs_gbr2.score(X_test_ct, y_test)

print(f'gs_gbr2_R2_train: {round(gs_gbr2_R2_train, 4)}')
print(f'gs_gbr2_R2_test: {round(gs_gbr2_R2_test, 4)}')
print()

gs_gbr2_mse_train = metrics.mean_squared_error(y_train, gs_gbr2_pred_train)
gs_gbr2_mse_test = metrics.mean_squared_error(y_test, gs_gbr2_pred_test)

print(f'gs_gbr2_mse_train: {round(gs_gbr2_mse_train, 4)}')
print(f'gs_gbr2_mse_test: {round(gs_gbr2_mse_test, 4)}')

gs_gbr2_perf_dict = {
    'R2': [gs_gbr2_R2_train, gs_gbr2_R2_test],
    'MSE': [gs_gbr2_mse_train, gs_gbr2_mse_test]
}

gs_gbr2_perf_df = pd.DataFrame(gs_gbr2_perf_dict, index=['Train','Test'])
gs_gbr2_perf_df.T

gs_gbr2_R2_train: 0.7131
gs_gbr2_R2_test: 0.5477

gs_gbr2_mse_train: 4867.5201
gs_gbr2_mse_test: 7919.9789


,Train,Test
R2,0.713145,0.547725
MSE,4867.520123,7919.978895


In [53]:
gs_gbr2.best_estimator_

GradientBoostingRegressor(max_depth=6, max_features=200, n_estimators=400,
                          random_state=42)

In [21]:
# pd.DataFrame(gs_gbr2.cv_results_)

In [80]:
ct.get_feature_names()

/Users/anniewang/opt/anaconda3/envs/dsi-111/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


AttributeError: Transformer sc (type StandardScaler) does not provide get_feature_names.

In [73]:
ct.transformers_[1][1].get_feature_names().shape

(1213,)

In [76]:
gs_gbr2.best_estimator_.feature_importances_.shape

(1219,)

In [72]:
feature_imp = pd.DataFrame(gs_gbr2.best_estimator_.feature_importances_,
             columns=ct.transformers_[1][1].get_feature_names())
feature_imp

ValueError: Shape of passed values is (1219, 1), indices imply (1219, 1213)

##### Pickle and save the model

In [25]:
with open('../Models/totalcomp_gs_gbr2.pkl', 'wb') as f:
    pickle.dump(gs_gbr2, f)

In [26]:
with open('../Models/totalcomp_gs_gbr2.pkl', 'rb') as f:
    gs_gbr2_input = pickle.load(f)

In [27]:
new_data = {
    'company': 'Google', 
    'title': 'Data Scientist', 
    'yearsofexperience': '1', 
    'yearsatcompany': '0.1', 
    'year': '2020', 
    'month': '9', 
    'state': 'California', 
    'inflation_rate': '0.3', 
    'inflation_rate_3mos': '0.2', 
    'employment_rate': '0.99', 
    'employment_rate_3mos': '0.98'
}

new_data_df = pd.DataFrame(new_data, index=[0])
new_data_df

,company,title,yearsofexperience,yearsatcompany,year,month,state,inflation_rate,inflation_rate_3mos,employment_rate,employment_rate_3mos
0,Google,Data Scientist,1,0.1,2020,9,California,0.3,0.2,0.99,0.98


In [29]:
new_data_df_ct = ct.transform(new_data_df)
print(new_data_df_ct.shape)

pred_gs_gbr2_new_data = gs_gbr2_input.predict(new_data_df_ct)
pred_gs_gbr2_new_data

(1, 1219)


array([182.64420446])

In [42]:
year = 2020
month = 9
inflation_rate = 0.014
inflation_rate_3mos = 0.006
employment_rate = 0.93582054649934
employment_rate_3mos = 0.8939803405002691 
company= 'Google'
title = 'Data Scientist'
state = 'Florida'
yearsofexperience = 1
yearsatcompany = 2

new_data = {
    'company': company, 
    'title': title, 
    'yearsofexperience': yearsofexperience, 
    'yearsatcompany': yearsatcompany, 
    'year': str(year), 
    'month': str(month), 
    'state': state, 
    'inflation_rate': inflation_rate, 
    'inflation_rate_3mos': inflation_rate_3mos, 
    'employment_rate': employment_rate, 
    'employment_rate_3mos': employment_rate_3mos
}


        
new_data_df = pd.DataFrame(new_data, index=[0])
new_data_df

,company,title,yearsofexperience,yearsatcompany,year,month,state,inflation_rate,inflation_rate_3mos,employment_rate,employment_rate_3mos
0,Google,Data Scientist,1,2,2020,9,Florida,0.014,0.006,0.935821,0.89398


In [33]:
st.sidebar.header('Total compensation Predictor')
select = st.sidebar.selectbox('Select Form', ['Form 1'], key='1')
if not st.sidebar.checkbox("Hide", True, key='1'):
    st.title('Fill in the following information:')
    company_name = st.selectbox(label = 'Company name:', options = df[company].unique())
    title = st.selectbox(label = 'Position title:', options = df[title].unique())
    years_of_experience = st.number_input("Years of experience:")
    years_at_company =  st.number_input("Years at company:")
    state = st.selectbox(label = 'US State:', options = df[state_short].unique())
submit = st.button('Predict')
if submit:
        
#new_data_df = pd.DataFrame([company_name, title, years_of_experience, years_at_company, year, month, state, inflation_rate,employment_rate,employment_rate_3mos])


SyntaxError: unexpected EOF while parsing (<ipython-input-33-d7da5b32267d>, line 13)

In [43]:
new_data_df_ct = ct.transform(new_data_df)
        
prediction = gs_gbr2.predict(new_data_df_ct)
#st.write('You can expect a total compensation of' prediction '$.')
prediction

array([129.41829369])

## Sileshi

## Mason